# Extract Airbnb Rates
----

#### Note
* Dataset request from rapidApi/Mashvisor endpoint rental rates

In [1]:
# Interactive plots
%matplotlib notebook

# Dependencies and Setup
import requests
import random
import json
import ast
import os
import datetime
import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
#import scipy.stats as st
#from citipy import citipy
#from pprint import pprint

# Set by selecting a datasource import type (CSV or API)
source = 'api'
#source = 'csv'

# Remember to update the config file with your API key
from api_keys import mash_api_key

# Set variable for current date using datetime library
today = datetime.date.today()

# Set Mashvisor API URL for rental rates
url = "https://mashvisor-api.p.rapidapi.com/rental-rates"

# Set API variables
state = "TX"
city = "Houston"
source = "airbnb"

# Set todays date as active and formatted as mm/dd/yy for API import
date = today.strftime('%m/%d/%y')
datef = today.strftime('%m%d%y')

# Set a specific date as active parameter for CSV import
#date = '09/12/20'
#datef = '091220'

# Print active date
print(date)
#print(mash_api_key)

12/02/20


In [2]:
# Create and call Mashvisor API query string for rental rates
querystring = {"state":state,"source":source,"city":city}
headers = {
    'x-rapidapi-key': mash_api_key,
    'x-rapidapi-host': "mashvisor-api.p.rapidapi.com"
    }
response = requests.request("GET", url, headers=headers, params=querystring)
print(response.text)

{"status":"success","content":{"retnal_rates":{"studio_value":913,"one_room_value":1103,"two_room_value":1337,"three_room_value":2210.5,"four_room_value":3374},"sample_count":1609,"detailed":[{"state":"TX","city":"Houston","neighborhood":null,"zipcode":null,"beds":"0","count":81,"min":0,"max":2512,"avg":999.0246913580247,"median":913,"adjusted_rental_income":1287.638888888889,"median_night_rate":67,"median_occupancy":53},{"state":"TX","city":"Houston","neighborhood":null,"zipcode":null,"beds":"1","count":645,"min":0,"max":3270,"avg":1095.0852713178294,"median":1103,"adjusted_rental_income":1309.7416666666666,"median_night_rate":72,"median_occupancy":51},{"state":"TX","city":"Houston","neighborhood":null,"zipcode":null,"beds":"2","count":413,"min":0,"max":5230,"avg":1435.3825665859565,"median":1337,"adjusted_rental_income":1505.6249999999998,"median_night_rate":100,"median_occupancy":45},{"state":"TX","city":"Houston","neighborhood":null,"zipcode":null,"beds":"3","count":318,"min":0,"ma

In [3]:
airdata = response.json()
print(airdata)

{'status': 'success', 'content': {'retnal_rates': {'studio_value': 913, 'one_room_value': 1103, 'two_room_value': 1337, 'three_room_value': 2210.5, 'four_room_value': 3374}, 'sample_count': 1609, 'detailed': [{'state': 'TX', 'city': 'Houston', 'neighborhood': None, 'zipcode': None, 'beds': '0', 'count': 81, 'min': 0, 'max': 2512, 'avg': 999.0246913580247, 'median': 913, 'adjusted_rental_income': 1287.638888888889, 'median_night_rate': 67, 'median_occupancy': 53}, {'state': 'TX', 'city': 'Houston', 'neighborhood': None, 'zipcode': None, 'beds': '1', 'count': 645, 'min': 0, 'max': 3270, 'avg': 1095.0852713178294, 'median': 1103, 'adjusted_rental_income': 1309.7416666666666, 'median_night_rate': 72, 'median_occupancy': 51}, {'state': 'TX', 'city': 'Houston', 'neighborhood': None, 'zipcode': None, 'beds': '2', 'count': 413, 'min': 0, 'max': 5230, 'avg': 1435.3825665859565, 'median': 1337, 'adjusted_rental_income': 1505.6249999999998, 'median_night_rate': 100, 'median_occupancy': 45}, {'sta

In [4]:
with open(f'../data/mash-api/{datef}/airbnb-{city}-{datef}.json', 'w') as f1:
    json.dump(airdata,f1)
    print("The airbnb json file is created")
f1.close()

The airbnb json file is created


In [5]:
f = open(f'../data/mash-api/{datef}/airbnb-{city}-{datef}.json', 'r')
s = f.read()
s = s.replace("\'", "\"")
s = s.replace("null", "''")
airstr = json.dumps(s)
airjson = json.loads(airstr)
airbnb = ast.literal_eval(airjson)
df = pd.DataFrame(airbnb)
print(df)
print("")
samples = df.iloc[2,1]
#print(samples)
#print("")
dict1 = {'median_rate': {'sample_count':samples}}
df_sample = pd.DataFrame(dict1)
#print(df_sample)
df_rates = pd.DataFrame([df.iloc[1,1]])
df_rates = df_rates.T
df_rates = df_rates.rename(columns = {0: "median_rate"})
df_rates2 = pd.concat([df_rates,df_sample],axis=0) 
print(df_rates2)
df_detail = pd.DataFrame(df.iloc[0,1])
df_detail = df_detail.replace("''",'')
df_detail

               status                                            content
detailed      success  [{'state': 'TX', 'city': 'Houston', 'neighborh...
retnal_rates  success  {'studio_value': 913, 'one_room_value': 1103, ...
sample_count  success                                               1609

                  median_rate
studio_value            913.0
one_room_value         1103.0
two_room_value         1337.0
three_room_value       2210.5
four_room_value        3374.0
sample_count           1609.0


,state,city,neighborhood,zipcode,beds,count,min,max,avg,median,adjusted_rental_income,median_night_rate,median_occupancy
0,TX,Houston,,,0,81,0,2512,999.024691,913.0,1287.638889,67.0,53
1,TX,Houston,,,1,645,0,3270,1095.085271,1103.0,1309.741667,72.0,51
2,TX,Houston,,,2,413,0,5230,1435.382567,1337.0,1505.625000,100.0,45
3,TX,Houston,,,3,318,0,7775,2384.018868,2210.5,2397.847222,150.0,50
4,TX,Houston,,,4,114,0,17712,3862.543860,3374.0,3934.700000,236.0,54
5,TX,Houston,,,5,21,293,12258,4707.285714,3229.0,4421.873611,279.0,52
6,TX,Houston,,,6,8,0,8035,3127.500000,2556.5,3482.708333,300.0,39
7,TX,Houston,,,7,3,3238,11635,6879.000000,5764.0,2670.684722,777.0,14
8,TX,Houston,,,8,1,12762,12762,12762.000000,12762.0,11933.066667,988.0,42
9,TX,Houston,,,12,1,17955,17955,17955.000000,17955.0,17364.875000,810.0,72


In [6]:
result = df_detail.to_json(orient="records")
parsed = json.loads(result)
detail_json = json.dumps(parsed, indent=4)
print(detail_json)

[
    {
        "state": "TX",
        "city": "Houston",
        "neighborhood": "",
        "zipcode": "",
        "beds": "0",
        "count": 81,
        "min": 0,
        "max": 2512,
        "avg": 999.024691358,
        "median": 913.0,
        "adjusted_rental_income": 1287.6388888889,
        "median_night_rate": 67.0,
        "median_occupancy": 53
    },
    {
        "state": "TX",
        "city": "Houston",
        "neighborhood": "",
        "zipcode": "",
        "beds": "1",
        "count": 645,
        "min": 0,
        "max": 3270,
        "avg": 1095.0852713178,
        "median": 1103.0,
        "adjusted_rental_income": 1309.7416666667,
        "median_night_rate": 72.0,
        "median_occupancy": 51
    },
    {
        "state": "TX",
        "city": "Houston",
        "neighborhood": "",
        "zipcode": "",
        "beds": "2",
        "count": 413,
        "min": 0,
        "max": 5230,
        "avg": 1435.382566586,
        "median": 1337.0,
        "adjus

In [7]:
result = df_rates2.to_json(orient="index")
parsed = json.loads(result)
rates_json = json.dumps(parsed, indent=4)
print(rates_json)

{
    "studio_value": {
        "median_rate": 913.0
    },
    "one_room_value": {
        "median_rate": 1103.0
    },
    "two_room_value": {
        "median_rate": 1337.0
    },
    "three_room_value": {
        "median_rate": 2210.5
    },
    "four_room_value": {
        "median_rate": 3374.0
    },
    "sample_count": {
        "median_rate": 1609.0
    }
}


In [8]:
with open(f'../data/mash-api/{datef}/airbnb-{city}-rates-{datef}.json', 'w') as f1:
    json.dump(rates_json,f1,indent=3)
    print("The rates json file is created")
f1.close()
with open(f'../data/mash-api/{datef}/airbnb-{city}-detailed-{datef}.json', 'w') as f2:
    json.dump(detail_json,f2,indent=4)
    print("The detailed json file is created")
f2.close()

The rates json file is created
The detailed json file is created


In [9]:
# END JSON PARSE FROM API